# Importing Packages

In [1]:
import pandas as pd
import numpy as np
import math
from datetime import date, datetime, timedelta
from sqlalchemy import create_engine, text
import os
import json
import pymysql
import psycopg2
from tqdm import tqdm
import warnings

In [3]:
group_num_df = pd.read_pickle('.pkl')
group_num_df = group_num_df[['user_id', 'group_id', 'source']]

In [4]:
group_num_df.loc[group_num_df['user_id'].isin(['UCS1uQ6B1xwbwhht7hulTJoA', 
                                               'UC-h957oPk16yoxl6IhDbu6w'])]

,user_id,group_id,source
67170,UC-h957oPk16yoxl6IhDbu6w,71249,youtube
67171,UCS1uQ6B1xwbwhht7hulTJoA,71249,youtube


# Connecting to DB of Infodemic

In [5]:
class EventDBConnector:

    def __init__(self):
        db_config = {
            'host': '',
            'username': '',
            'password': '',
            'port': 5432,
            'database': ''
        }
        
        ssl_options = {
            'sslmode': 'require',  # 設定為 'require' 表示要求啟用 SSL
            'sslrootcert': '/path/to/ca-certificate.crt',  # 可選的 CA 證書路徑，根據你的需要設定
            # 如果你有 client 證書，也可以設定以下選項
            # 'sslcert': '/path/to/client-certificate.crt',
            # 'sslkey': '/path/to/client-certificate.key',
        }
        
        engine_str = f"postgresql+psycopg2://%(username)s:%(password)s@%(host)s:%(port)s/%(database)s" % db_config
        self.engine = create_engine(engine_str, connect_args=ssl_options)

    def query(self, query):
        conn = self.engine.connect()
        df = pd.read_sql_query(text(query), conn)
        conn.close()
        return df

In [6]:
class EntityDBConnector:

    def __init__(self):
        self.db_config = {
            'host': '',
            'user': '',
            'password': '',
            'port': 3306
        }

    def query(self, query):
        conn = pymysql.connect(**self.db_config)
        res = pd.read_sql(query, conn)
        conn.close()
        return res

# Defining Functions for Data Table

In [7]:
def get_platform_entity(content_hash):

    entity_db_connector = EntityDBConnector()

    sql_query = '''
    SELECT *
    FROM `collab`.operation_sentiment 
    WHERE content_hash IN {}
    AND sentiment >= 2
    '''
    entity_df = entity_db_connector.query(sql_query.format(tuple(content_hash)))
    entity_df = entity_df.loc[entity_df['sentiment'] >= 2]
    return entity_df

In [223]:
event_db_connector = EventDBConnector()

sql_query = '''
SELECT * FROM events
WHERE event_time > '2023-10-13 23:59:59'
AND event_time < '2023-10-21 23:59:59';
'''

df = event_db_connector.query(sql_query)

In [222]:
df

,event_id,story_id,event_time,created_at,updated_at,deleted_at,language,categories,hashtags,image,title,summary_short,summary_long,count,meta,event_times
0,05cee1cce2b8,,2023-10-15 18:15:55+00:00,2023-10-17 08:42:21+00:00,2023-10-17 08:42:21+00:00,NaT,zh-hant,[entertainment],None,,"{'en': 'Magic tool for making dumplings', 'zh_hans': '包饺子神器', 'zh_hant': '包餃子神器'}","[{'en': 'I am a native of northern China. I especially like to eat and make dumplings. When I was a child, my mother always led the whole family to make dumplings. I also got a small piece of dough and learned to roll the dumpling skin and make dumplings.', 'zh_hans': '我是中国北方人，特别喜欢吃饺子、包饺子，小时候母亲总是领著全家人包饺子，我也会分得一小块面团，学擀饺皮及包饺子。', 'zh_hant': '我是中國北方人，特別喜歡吃餃子、包餃子，小時候母親總是領著全家人包餃子，我也會分得一小塊麵團，學擀餃皮及包餃子。'}, {'en': 'Mother is a master of making dumplings, and she keeps noodles and rolling dumplings, which she makes.', 'zh_hans': '母亲是包饺子高手，和面、擀饺皮都拿手，她包出的。', 'zh_hant': '母親是包餃子高手，和麵、擀餃皮都拿手，她包出的。'}, {'en': 'The patterns of dumplings include crescent dumplings, Yuanbao dumplings, sunflower dumplings and mouse dumplings. Folk talk of mouse dumplings have the meanings of ""rats bite the sky open"" and ""make an auspicious fortune.""', 'zh_hans': '饺子的花样有月牙饺子、元宝饺子、葵花饺子以及老鼠饺子，民间讲老鼠饺子有「鼠咬天开」、「吉祥发财」等寓意。', 'zh_hant': '餃子的花樣有月牙餃子、元寶餃子、葵花餃子以及老鼠餃子，民間講老鼠餃子有「鼠咬天開」、「吉祥發財」等寓意。'}, {'en': 'The dumpling skin rolled out...","[{'en': 'I am a native of northern China. I especially like to eat and make dumplings. When I was a child, my mother always led the whole family to make dumplings. I also got a small piece of dough and learned to roll the dumpling skin and make dumplings.', 'zh_hans': '我是中国北方人，特别喜欢吃饺子、包饺子，小时候母亲总是领著全家人包饺子，我也会分得一小块面团，学擀饺皮及包饺子。', 'zh_hant': '我是中國北方人，特別喜歡吃餃子、包餃子，小時候母親總是領著全家人包餃子，我也會分得一小塊麵團，學擀餃皮及包餃子。'}, {'en': 'Mother is a master of making dumplings, and she keeps noodles and rolling dumplings, which she makes.', 'zh_hans': '母亲是包饺子高手，和面、擀饺皮都拿手，她包出的。', 'zh_hant': '母親是包餃子高手，和麵、擀餃皮都拿手，她包出的。'}, {'en': 'Thin filling is too much, after cooking is a mouthful of full dumplings.', 'zh_hans': '薄馅多，煮好后是一口一个的饱满水饺。', 'zh_hant': '薄餡多，煮好後是一口一個的飽滿水餃。'}, {'en': 'After graduating from a foreign language university in China in 1983, I worked in a university, was responsible for the management of foreign cultural and educational experts and took care of their lives. I invited them to visit my house every ye...",{'news': 1},{},"[2023-10-15 18:15:55+00:00, 2023-10-15 18:15:55+00:00]"
1,4fdebf9dcf8d,sf8d2de74b30,2023-10-16 03:01:14+00:00,2023-10-16 15:42:30+00:00,2023-10-17 09:43:24.462331+00:00,NaT,zh-hant,"[world, others]",None,https://attach.setn.com/newsimages/2023/09/21/4331614-PH.jpg,"{'en': 'The Russia-Ukraine war enters its 600th day, and both sides claim to have their own upper hand', 'zh_hans': '俄乌战争迈入第600天双方皆称自己占上风', 'zh_hant': '俄烏戰爭邁入第600天雙方皆稱自己占上風'}","[{'en': 'Russia's aggression against Ukraine entered its 600th day today (16th). The two sides continued to engage in heavy fighting, claiming that they were at an advantage in military matters.', 'zh_hans': '俄罗斯对乌克兰的侵略今（16）日迈入第600天，双方持续进行激战，皆声称己方在军事方面处于优势。', 'zh_hant': '俄羅斯對烏克蘭的侵略今（16）日邁入第600天，雙方持續進行激戰，皆聲稱己方在軍事方面處於優勢。'}, {'en': 'Ukrainian President Volodymyr Zelenskyy gave a speech to the nation as the full-scale war between Russia and Ukraine entered its 600th day, thanking everyone who fought for the country's freedom and worked hard, Ukrinform reported.', 'zh_hans': '乌克兰国家通讯社（Ukrinform）报导，在俄乌全面战争进入第600天之际，乌克兰总统泽伦斯基（Volodymyr Zelenskyy）对国民发表演说，感谢每一位为国家自由奋斗及努力的人。', 'zh_hant': '烏克蘭國家通訊社（Ukrinform）報導，在俄烏全面戰爭進入第600天之際，烏克蘭總統澤倫斯基（Volodymyr Zelenskyy）對國民發表演說，感謝每一位為國家自由奮鬥及努力的人。'}]","[{'en': 'Russia's aggression against Ukraine entered its 600th day today (16th). The two sides continued to engage in heavy fighting, claiming that they were at an advantage in military matters.', 'zh_hans': '俄罗斯对乌克兰的侵略今（16）日迈入第600天，双方持续进行激战，皆声称己方在军事方面处于优势。', 'zh_hant': '俄羅斯對烏克蘭的侵略今（16）日邁入第600天，雙方持續進行激戰，皆聲稱己方在軍事方面處於優勢。'}, {'en': 'Ukrainian President Volodymyr Zelenskyy gave a

In [224]:
event_db_connector = EventDBConnector()

sql_query = '''
SELECT * FROM event_social_counts
WHERE event_id IN {};
'''

social_count_df = event_db_connector.query(sql_query.format(tuple(df['event_id'].unique().tolist())))
social_count_df

,event_id,platform,updated_at,post,buzz,collab_buzz,collab_account,collab_group,meta
0,e1e8baf6669b,twitter,2023-10-20 06:13:35.172901+00:00,1,5,0,0,0,{}
1,e1e8baf6669b,all,2023-10-20 06:13:35.172901+00:00,1,5,0,0,0,{}
2,ed2006f6bdbf,twitter,2023-10-20 06:13:36.019050+00:00,2,13,0,0,0,{}
3,ed2006f6bdbf,all,2023-10-20 06:13:36.019050+00:00,2,13,0,0,0,{}
4,f067644fd7a1,youtube,2023-10-21 10:13:28.258106+00:00,1,1,0,0,0,{}
...,...,...,...,...,...,...,...,...,...
108385,df151b98e7f3,all,2023-10-21 21:13:43.164242+00:00,2,9,0,0,0,{}
108386,e43fbf950469,ptt,2023-10-23 05:13:19.658280+00:00,1,262,20,16,0,{}
108387,e43fbf950469,all,2023-10-23 05:13:19.658280+00:00,5,321,21,17,0,{}
108388,db3f9e66ca45,twitter,2023-10-23 00:13:44.562959+00:00,4,36,1,1,0,{}


# Pro-processing Data Set

In [225]:
event_to_title = {}
event_to_summary = {}
event_to_story = {}
event_to_time = {}

event_to_title = {event_id : zh_title for event_id, zh_title in zip(df['event_id'], df['title'])}
event_to_story = {event_id : story_id for event_id, story_id in zip(df['event_id'], df['story_id'])}
event_to_time = {event_id : event_time for event_id, event_time in zip(df['event_id'], df['event_time'])}

for event_id, summary in zip(df['event_id'], df['summary_short']):
    event_to_summary[event_id] = summary[0]['zh_hant']

In [226]:
social_all_df = social_count_df.loc[social_count_df['platform'] == 'all'].reset_index(drop = True)
social_sep_df = social_count_df.loc[social_count_df['platform'] != 'all'].reset_index(drop = True)

In [229]:
fb_df = social_count_df.loc[social_count_df['platform'] == 'facebook'].reset_index(drop = True)
yt_df = social_count_df.loc[social_count_df['platform'] == 'youtube'].reset_index(drop = True)
ptt_df = social_count_df.loc[social_count_df['platform'] == 'ptt'].reset_index(drop = True)
ttr_df = social_count_df.loc[social_count_df['platform'] == 'twitter'].reset_index(drop = True)
ttk_df = social_count_df.loc[social_count_df['platform'] == 'tiktok'].reset_index(drop = True)

fb_dict = {event_id : collab_buzz for event_id, collab_buzz in zip(fb_df['event_id'], fb_df['collab_buzz'])}
yt_dict = {event_id : collab_buzz for event_id, collab_buzz in zip(yt_df['event_id'], yt_df['collab_buzz'])}
ptt_dict = {event_id : collab_buzz for event_id, collab_buzz in zip(ptt_df['event_id'], ptt_df['collab_buzz'])}
ttr_dict = {event_id : collab_buzz for event_id, collab_buzz in zip(ttr_df['event_id'], ttr_df['collab_buzz'])}
ttk_dict = {event_id : collab_buzz for event_id, collab_buzz in zip(ttk_df['event_id'], ttk_df['collab_buzz'])}

In [232]:
stat_df = social_sep_df.groupby(['event_id']).agg({'collab_buzz':'sum'}).reset_index().sort_values('collab_buzz', ascending = False)
stat_df

,event_id,collab_buzz
19036,c99e25a2de2f,6273
16948,af323a348374,6267
21300,e8254d5cdafa,4935
15922,a264f6a63afb,3272
9874,5d658d6e893a,3244
...,...,...
9731,5bcb854a605c,0
9732,5bcea9eaa24d,0
9733,5bd02d2a6110,0
9735,5bd189a73bcb,0


In [233]:
stat_df = stat_df.reset_index(drop = True)
stat_df

,event_id,collab_buzz
0,c99e25a2de2f,6273
1,af323a348374,6267
2,e8254d5cdafa,4935
3,a264f6a63afb,3272
4,5d658d6e893a,3244
...,...,...
23055,5bcb854a605c,0
23056,5bcea9eaa24d,0
23057,5bd02d2a6110,0
23058,5bd189a73bcb,0


In [235]:
# warnings.filterwarnings('ignore', category = SettingWithCopyWarning)

stat_df['event_summary'] = stat_df['event_id'].apply(lambda x: event_to_summary[x])
stat_df['story_id'] = stat_df['event_id'].apply(lambda x: event_to_story[x])
stat_df['event_time'] = stat_df['event_id'].apply(lambda x: event_to_time[x])

stat_df['facebook'] = stat_df['event_id'].apply(lambda x:fb_dict.get(x, 0))
stat_df['youtube'] = stat_df['event_id'].apply(lambda x:yt_dict.get(x, 0))
stat_df['ptt'] = stat_df['event_id'].apply(lambda x:ptt_dict.get(x, 0))
stat_df['twitter'] = stat_df['event_id'].apply(lambda x:ttr_dict.get(x, 0))
stat_df['tiktok'] = stat_df['event_id'].apply(lambda x:ttk_dict.get(x, 0))
stat_df['zh_title'] = stat_df['event_id'].apply(lambda x:event_to_title[x]['zh_hant'] if 'zh_hant' in event_to_title[x] else '')
stat_df['en_title'] = stat_df['event_id'].apply(lambda x:event_to_title[x]['en'] if 'en' in event_to_title[x] else '')

# stat_df = stat_df.loc[~stat_df['zh_title'].str.contains('航海')]

stat_df = stat_df[['event_id', 'event_time', 
                   'story_id', 'zh_title', 'en_title', 
                   'event_summary', 'collab_buzz', 
                   'facebook', 'youtube', 'ptt', 'twitter', 'tiktok']]

fb_sum = stat_df['facebook'].sum()
yt_sum = stat_df['youtube'].sum()
ptt_sum = stat_df['ptt'].sum()
ttr_sum = stat_df['twitter'].sum()
ttk_sum = stat_df['tiktok'].sum()

stat_df['facebook_percent'] = stat_df['facebook'].apply(lambda x:np.around(x/fb_sum, 3)*100)
stat_df['youtube_percent'] = stat_df['youtube'].apply(lambda x:np.around(x/yt_sum, 3)*100)
stat_df['ptt_percent'] = stat_df['ptt'].apply(lambda x:np.around(x/ptt_sum, 3)*100)
stat_df['twitter_percent'] = stat_df['twitter'].apply(lambda x:np.around(x/ttr_sum, 3)*100)
stat_df['tiktok_percent'] = stat_df['tiktok'].apply(lambda x:np.around(x/ttk_sum, 3)*100)

stat_df

/tmp/ipykernel_2346/550601028.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stat_df['facebook_percent'] = stat_df['facebook'].apply(lambda x:np.around(x/fb_sum, 3)*100)
/tmp/ipykernel_2346/550601028.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stat_df['youtube_percent'] = stat_df['youtube'].apply(lambda x:np.around(x/yt_sum, 3)*100)
/tmp/ipykernel_2346/550601028.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

,event_id,event_time,story_id,zh_title,en_title,event_summary,collab_buzz,facebook,youtube,ptt,twitter,tiktok,facebook_percent,youtube_percent,ptt_percent,twitter_percent,tiktok_percent
0,c99e25a2de2f,2023-10-16 04:42:00+00:00,s35db3ec1f29,華春瑩阿語發文：上海曾庇護2萬猶太人相信己所不欲，勿施於人,"Hua Chun-ying posted in Arabic: Shanghai once sheltered 20,000 Jews who believed they did not want to do anything to others",中國外交部長助理兼新聞司長華春瑩15日在社群平台X上，以英語、阿拉伯語發文，重提二戰時上海曾為2萬名猶太難民提供庇護的歷史，過去的悲劇不應發生在今天的任何人身上。,6273,257,3,6013,0,0,0.5,0.1,2.0,0.0,0.0
1,af323a348374,2023-10-16 04:21:19+00:00,s0b9d61d07a5,影/藍白合方案2選1？黃珊珊拒開放式初選金溥聰重話回應：要折衷,Shadow / Blue and White Plan 2 choose 1? Huang Shan-shan refused to respond to Chin Pu-tsung in the open primary election: Compromise,國民黨總統參選人競辦執行長金溥聰，14日與民眾黨總統參選人柯文哲競辦總幹事黃珊珊洽談「藍白合」，拋出開放式初選方案，黃珊珊今（17）日表示不可行，金溥聰再回應喊失望，侯辦則澄清稱，金溥聰並未說沒有下一步。,6267,411,13,5843,0,0,0.8,0.3,2.0,0.0,0.0
2,e8254d5cdafa,2023-10-16 03:54:04+00:00,sb3741433fa2,以色列進軍在即！400車輛集結加薩北部附近衛星照曝光,Israel's march is imminent! 400 vehicles assembled near northern Gaza satellite photos exposed,衛星影像分析顯示，約400台車輛已集結在以色列境內觸及巴勒斯坦領土加薩走廊北部的地點，顯示以色列可能正趕緊在邊境10公里範圍內部署武力，準備發動地面入侵。,4935,42,0,4893,0,0,0.1,0.0,1.7,0.0,0.0
3,a264f6a63afb,2023-10-16 06:43:16+00:00,s61fea7da9b0,侯友宜：盡力讓藍白合往前推 朝政黨輪替往前走,Hou Yu-yi: Try your best to push the blue and white together forward and move forward towards the rotation of political parties,國民黨與民眾黨週末兩天針對「藍白合」舉行會談，但雙方卻談出火氣，甚至柯文哲還怒嗆「我讓給你選你也不會上！」對此，侯友宜競辦發言人黃子哲今（16）日喊話「妥協」，表示目前沒有看到完全破局的狀況，大家過程中難免會有不同意見，期盼能更心平氣和、創造良好氛圍，也呼籲民眾黨必須坐下來談，才有機會達成妥協。,3272,146,0,3126,0,0,0.3,0.0,1.1,0.0,0.0
4,5d658d6e893a,2023-10-17 02:38:00+00:00,sec89f7fabf9,藍白同場避見面！侯喊「利他」柯虧放輕鬆：別每天拿高帽子來戴,"Blue and white avoid meeting in the same field! Hou shouted ""Lita"" and Ke Yu took it easy: Don't wear high hats every day",2024總統大選「藍白合」暫時陷入僵局，雙方對決定在野最強候選人的方式缺乏共識，國民黨提出民調、民主初選並行的折衷方案，總統參選人侯友宜昨（18）日更以「切豬肉」比喻。,3244,187,16,3041,0,0,0.3,0.3,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23055,5bcb854a605c,2023-10-17 11:01:18+00:00,s0daff45b2bf,57歲男子被發現死在皇后區紐約地鐵車廂內,57-year-old man found dead on NYC subway car in Queens,據警方稱，週一晚上，一名57歲的男子被發現死在皇后區一條火車線末端的地鐵車廂內。,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
23056,5bcea9eaa24d,2023-10-20 01:11:36+00:00,s3a34326c84d,中方是否參加中日韓外長會議？外交部回應,Does China participate in the China-Japan-South Korea foreign ministers meeting? Ministry of Foreign Affairs responds,日本共同社20日報道稱，據多名外交消息人士19日透露，圍繞中日韓三國正在協調的外長會談，主席國韓國就11月26日前後實施的日程徵詢了中日兩國的意見。,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
23057,5bd02d2a6110,2023-10-15 03:07:00+00:00,,他幫柯文哲唱出真實心聲網友噴笑狂讚：神曲！！！,"He helped Ke Wen-che sing his true heart, netizens laughed and praised: Divine Comedy!!!",《台灣迷因taiwan meme》粉絲團分享《迷の曲：藍白合》影片，原曲是周興哲的《怎麼了》，作詞是咪咪老師，演唱是真O柯文哲，粉專小編並寫道：「幫柯文哲唱出他對藍白合的真實心聲，也唱出他對辣個男人的思念...。」。,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
23058,5bd189a73bcb,2023-10-17 16:30:00+00:00,,你應該有一個家庭「安全的話」來反對AI語音欺騙詐騙嗎？,Should you have a family ‘safe word’ against AI voice-spoofing scams?,大多數這些騙局涉及有人冒充企業或官方機構索要金錢或個人資訊。廉價且易於使用的ai還讓犯罪分子變得非常簡單，可以偽造陷入困境的孫子的聲音，或者一個孩子似乎想說什麼就說什麼。,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0


In [194]:
stat_df.groupby('story_id').agg({'collab_buzz':'sum'}).reset_index().sort_values('collab_buzz', ascending = False).head(10)

,story_id,collab_buzz
0,,21792
2333,s60c7fcdf752,14446
275,s0b9d61d07a5,13025
698,s1d629b5d932,10606
1311,s35db3ec1f29,7482
4360,sb3741433fa2,4942
5756,sec89f7fabf9,4130
1465,s3bd53d08b9e,3493
2364,s61fea7da9b0,3490
5408,sdd0e5d8fd71,2527


# Setting Keywords for searching Maniulation Amount of Issues

In [200]:
stat_df.groupby(['event_id', 'zh_title'])['collab_buzz'].sum().to_frame().reset_index().sort_values('collab_buzz', ascending = False).head(20)

,event_id,zh_title,collab_buzz
17300,c99e25a2de2f,華春瑩阿語發文：上海曾庇護2萬猶太人相信己所不欲，勿施於人,6273
15397,af323a348374,影/藍白合方案2選1？黃珊珊拒開放式初選金溥聰重話回應：要折衷,6267
19341,e8254d5cdafa,以色列進軍在即！400車輛集結加薩北部附近衛星照曝光,4935
14474,a264f6a63afb,侯友宜：盡力讓藍白合往前推 朝政黨輪替往前走,3272
8937,5d658d6e893a,藍白同場避見面！侯喊「利他」柯虧放輕鬆：別每天拿高帽子來戴,3244
2318,154ef6f7337d,以巴衝突 | 揭哈馬斯神秘「地下武器」面紗 加沙迷宮傳延綿500公里、助避過鐵穹導彈,2110
8317,5688d0dd45aa,埃及允20車救援物資進加沙,2110
470,043aeb51e3d9,柯文哲喊「藍白合」比民調！全台僅1縣市不挺還是他最熟悉的,2032
5024,3137312adf5c,「供以色列一切所需」拜登宣布供加薩與西岸一億美金,1884
13449,94b041b81a7d,「藍白合」歧見！金溥聰批柯說翻臉就翻臉「全民調」怎選最強候選人,1828


In [204]:
k1 = '加薩|加沙|Gaza|以巴|巴勒|巴以|以色列|哈瑪斯|哈馬斯|以總理|本雅明|Israel|Israli|Palestine|Hamas|Netanyahu|WestBank|West Bank|庇護2萬猶太人'
k2 = 'Lin Bay|LinBay|好油|超思|液蛋|蛋液|吉仲|巴西蛋|進口蛋|蛋進口|蛋價|畜產會|變質雞蛋|綠蛋|臭蛋'
k3 = '川普|特朗普|茱莉安尼|Trump'
k4 = '眾議院議長|麥卡錫'
k5 = '藍白必合|藍白合|藍白比|藍白同場'
k6 = '賴清德'
k7 = '澤連斯基|烏克蘭|澤倫斯基'
k8 = '蔡英文|蔡總統'
k9 = '拜登|Biden'
k10 = '佩霞|郭台銘|郭董|我姓郭'

k_lst = [k1, k2, k3, k4, k5, 
         k6, k7, k8, k9, k10]

stat_df_event = []

for i in k_lst:
    temp_stat = stat_df.loc[stat_df['zh_title'].str.contains(i)]
    temp_stat = temp_stat.reset_index(drop = True)
    
    event_keyword = i
    event_sum_title = temp_stat['zh_title'][0]
    event_sum_title_en = temp_stat['en_title'][0]
    
    fb_sum = temp_stat['facebook_percent'].sum()
    yt_sum = temp_stat['youtube_percent'].sum()
    ptt_sum = temp_stat['ptt_percent'].sum()
    ttr_sum = temp_stat['twitter_percent'].sum()
    ttk_sum = temp_stat['tiktok_percent'].sum()
    
    total_sum = np.around((fb_sum+yt_sum+ptt_sum+ttr_sum+ttk_sum)/5, 1)
        
    stat_df_event.append({'Event_keyword': event_keyword, 
                          'Event name': event_sum_title, 
                          'Event name EN': event_sum_title_en,
                          'Manipulation % from Troll Groups on Facebook': fb_sum, 
                          'Manipulation % from Troll Groups on YouTube': yt_sum, 
                          'Manipulation % from Troll Groups on Ptt': ptt_sum, 
                          'Manipulation % from Troll Groups on Twitter': ttr_sum, 
                          'Manipulation % from Troll Groups on Tiktok': ttk_sum,
                          'Overall Manipulation % from all Social Media': total_sum})

stat_df_event = pd.DataFrame(stat_df_event)
stat_df_event = stat_df_event.sort_values('Overall Manipulation % from all Social Media', 
                                          ascending = False).reset_index(drop = True)

stat_df_event['Accumualte % of Manipulation %'] = stat_df_event['Overall Manipulation % from all Social Media'].cumsum()
pd.set_option('max_colwidth', 1_000)
stat_df_event

,Event_keyword,Event name,Event name EN,Manipulation % from Troll Groups on Facebook,Manipulation % from Troll Groups on YouTube,Manipulation % from Troll Groups on Ptt,Manipulation % from Troll Groups on Twitter,Manipulation % from Troll Groups on Tiktok,Overall Manipulation % from all Social Media,Accumualte % of Manipulation %
0,加薩|加沙|Gaza|以巴|巴勒|巴以|以色列|哈瑪斯|哈馬斯|以總理|本雅明|Israel|Israli|Palestine|Hamas|Netanyahu|WestBank|West Bank|庇護2萬猶太人,華春瑩阿語發文：上海曾庇護2萬猶太人相信己所不欲，勿施於人,"Hua Chun-ying posted in Arabic: Shanghai once sheltered 20,000 Jews who believed they did not want to do anything to others",8.8,54.7,19.3,6.4,0.0,17.8,17.8
1,賴清德,賴清德轟登門帶訪說自導自演 柯文哲反嗆：我去把行程調出來對質,Lai Ching-te blasted the door and took the interview to say that he directed and acted himself. Ke Wen-che said: I will adjust the itinerary to confront,7.6,1.4,6.8,0.0,24.9,8.1,25.9
2,藍白必合|藍白合|藍白比|藍白同場,影/藍白合方案2選1？黃珊珊拒開放式初選金溥聰重話回應：要折衷,Shadow / Blue and White Plan 2 choose 1? Huang Shan-shan refused to respond to Chin Pu-tsung in the open primary election: Compromise,3.5,2.5,12.3,0.0,18.7,7.4,33.3
3,川普|特朗普|茱莉安尼|Trump,法官在聯邦大選案中對唐納·川普下達禁令,Judge places gag order on Donald Trump in federal election case,4.5,2.3,0.0,21.3,0.0,5.6,38.9
4,拜登|Biden,「供以色列一切所需」拜登宣布供加薩與西岸一億美金,"""For everything Israel needs"" Biden announces $100 million for Gaza and the West Bank",1.2,7.6,3.3,2.0,0.0,2.8,41.7
5,眾議院議長|麥卡錫,眾議院議長無人,Nobody for Speaker of the House,4.2,2.1,0.0,5.9,0.0,2.4,44.1
6,澤連斯基|烏克蘭|澤倫斯基,澤連斯基希望訪以色列 據報被告知時機不對,"Zelensky wants to visit Israel, reportedly told the timing was wrong",0.0,2.0,0.3,1.5,0.0,0.8,44.9
7,佩霞|郭台銘|郭董|我姓郭,國民黨拋民主初選 郭台銘：太深奧！要消化幾天,KMT throws democratic primary election Kuo Tai-ming: Too esoteric! A few days to digest,1.7,0.6,1.5,0.0,0.0,0.8,45.7
8,蔡英文|蔡總統,蔡總統：再生能源發電有成 10月太陽+風能發電量創高,"President Tsai: Renewable energy generation has reached a high in October, when the sun + wind power generation hit a high",3.3,0.1,0.2,0.0,0.0,0.7,46.4
9,Lin Bay|LinBay|好油|超思|液蛋|蛋液|吉仲|巴西蛋|進口蛋|蛋進口|蛋價|畜產會|變質雞蛋|綠蛋|臭蛋,好油恐嚇案主嫌是網軍頭「筆電內藏上千帳號」負責下指令帶風向,"The main suspect in the good oil intimidation case is the head of the cyber army, ""thousands of accounts hidden in the laptop"" in charge of the instructions and the wind direction",0.3,0.0,0.5,0.0,0.0,0.2,46.6


In [206]:
input_kw = k1

temp_stat = stat_df.loc[stat_df['zh_title'].str.contains(input_kw)|stat_df['event_summary'].str.contains(input_kw)]

fb_sum = temp_stat['facebook_percent'].sum()
yt_sum = temp_stat['youtube_percent'].sum()
ptt_sum = temp_stat['ptt_percent'].sum()
ttr_sum = temp_stat['twitter_percent'].sum()
total_sum = temp_stat['collab_buzz'].sum()

print('facebook: ', fb_sum)
print('youtube: ', yt_sum)
print('ptt: ', ptt_sum)
print('twitter: ', ttr_sum)
print('tiktok: ', ttk_sum)
print('total: ', np.around(((fb_sum+yt_sum+ptt_sum+ttr_sum+ttk_sum)/4), 3))
temp_stat.sort_values('collab_buzz', ascending = False).head(10)

temp_stat.groupby('story_id').agg({'collab_buzz':'sum'}).reset_index().sort_values('collab_buzz', ascending = False).head(10)
temp_stat.groupby('event_id').agg({'collab_buzz':'sum'}).reset_index().sort_values('collab_buzz', ascending = False).tail(10)

facebook:  10.6
youtube:  64.8
ptt:  20.100000000000005
twitter:  7.300000000000001
tiktok:  0.0
total:  25.7


,event_id,collab_buzz
982,8debca8387e4,0
983,8e3fbe9f7df2,0
109,0ca8087fee5a,0
985,8e7e3d1e8ebc,0
986,8ee999c6b748,0
987,8f0f493d2955,0
531,4830778ab5a5,0
108,0c866aa743dc,0
990,8fba48415dbf,0
901,7f4404bdbfc3,0


# Extracting the Entities of Events from Keywords

## Extracting the Entities

In [163]:
input_kw = k1
kw_list = stat_df.loc[stat_df['zh_title'].str.contains(input_kw)|stat_df['event_summary'].str.contains(input_kw)]['event_id'].unique().tolist()

## 1
social_id_connector = EntityDBConnector()

sql_query = '''
SELECT social_id, event_id, source, relevance_score FROM `event-clustering-v2`.social_event
WHERE event_id IN {};
'''

# social_id_df = social_id_connector.query(sql_query)
social_id_df = social_id_connector.query(sql_query.format(tuple(kw_list)))
# social_id_df = social_id_connector.query(sql_query.format(tuple(k2_list)))
social_id_df = social_id_df.groupby(['social_id','source']).apply(lambda v:v.sort_values('relevance_score', ascending=False).iloc[0])
social_id_df = social_id_df.reset_index(drop=True)

# 可能跟實際話題量有差異
social_id_df['source'].value_counts()
social_id_to_event = social_id_df.set_index('social_id')['event_id'].to_dict()
# social_id_df

## 2
entity_db_connector = EntityDBConnector()

sql_query = '''
SELECT social_id, user_id, platform, content_hash FROM `collab`.operation
WHERE social_id IN {};
'''

# entity_df_v2 = entity_db_connector.query(sql_query)
entity_df_v2 = entity_db_connector.query(sql_query.format(tuple(social_id_df['social_id'].tolist())))
# entity_df_v2 = entity_df_v2.loc[entity_df_v2['group_id'] > 60_000]
entity_df_v2 = pd.merge(entity_df_v2, group_num_df, on = 'user_id', how = 'inner')

entity_df_v2['event_id'] = entity_df_v2['social_id'].apply(lambda x:social_id_to_event[x])
hash_to_event = entity_df_v2.set_index('content_hash')['event_id'].to_dict()
# entity_df_v2

## 3
result = pd.DataFrame([])
aim_platform = entity_df_v2['platform'].unique()
aim_event = 'k1'

# for i in aim_platform:
for i in ['facebook', 'ptt', 'youtube', 'twitter']:
# for i in ['youtube']:
    i
    temp = entity_df_v2.loc[entity_df_v2['platform'] == i]
    temp = temp.loc[temp['content_hash'].notna()]
    
    temp_platform_entity = get_platform_entity(temp['content_hash'].tolist()+temp['content_hash'].tolist())
    temp_platform_entity['event_id'] = temp_platform_entity['content_hash'].apply(lambda v:hash_to_event[v])
    temp_platform_entity
    # for j in temp_platform_entity['event_id'].value_counts().index:
    #     'https://infodemic.cc/v2/en/event/'+j
    
    temp_platform_entity['sentiment'] = temp_platform_entity['sentiment'].apply(lambda v: ' [–]' if v == 3 else ' [＋]')
    
    temp_platform_entity['entity'] = temp_platform_entity['entity'] +temp_platform_entity['sentiment']
    temp_platform_entity = np.around(temp_platform_entity['entity'].value_counts(normalize = True)*100,2).to_frame('Normalized Manipulated Volume').reset_index()
    temp_platform_entity['Story_name'] = aim_event
    temp_platform_entity['platform'] = i
    temp_platform_entity = temp_platform_entity.rename(columns = {'index':'entity'})[['Story_name', 
                                                                                      'entity', 
                                                                                      'platform', 
                                                                                      'Normalized Manipulated Volume']]
    
    temp_platform_entity = temp_platform_entity.loc[temp_platform_entity['Normalized Manipulated Volume'] > 2]
    
    result = pd.concat([result, temp_platform_entity])

# result.to_csv('k6_event_sankey_diagram_data.csv', index = False)
result

/tmp/ipykernel_2346/1059091046.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  res = pd.read_sql(query, conn)
/tmp/ipykernel_2346/1059091046.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  res = pd.read_sql(query, conn)
/tmp/ipykernel_2346/1059091046.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  res = pd.read_sql(query, conn)
/tmp/ipykernel_2346/1059091046.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 obj

,Story_name,entity,platform,Normalized Manipulated Volume
0,k1,Israel [–],facebook,4.76
1,k1,Hamas [–],facebook,4.26
2,k1,以色列 [–],facebook,4.01
3,k1,Joe Biden [–],facebook,3.26
4,k1,巴勒斯坦 [–],facebook,2.76
5,k1,Palestine [–],facebook,2.76
6,k1,以色列 [＋],facebook,2.76
7,k1,哈瑪斯 [＋],facebook,2.01
0,k1,以色列 [–],ptt,10.45
1,k1,以色列 [＋],ptt,7.77


## Extracting the Troll Groups' Comments

In [165]:
comment_db_connector = EntityDBConnector()

sql_query = '''
SELECT *
FROM `collab`.operation_content
WHERE created_at > '2023-10-01 23:59:59'
AND skip_computing_sentiment = 0;
'''

comment_df = comment_db_connector.query(sql_query)

merged_comment_df = pd.merge(entity_df_v2, 
                             comment_df[['content', 'content_hash', 'created_at']], 
                             on = 'content_hash', 
                             how = 'inner')

len(merged_comment_df)
merged_comment_df

/tmp/ipykernel_2346/1059091046.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  res = pd.read_sql(query, conn)


,social_id,user_id,platform,content_hash,group_id,source,event_id,content,created_at
0,--95PN5JjGw,UCFi1gCWSp26jr93RptI-srw,youtube,d4a8dc455c5011d78f03eacfddf80f748992fb73f58a012ba49fd150e61e2c35,71353,youtube,788372699852,台巴子媒体没用客观，全是造谣。张口闭口反间谍法,2023-10-12 01:20:23.357413
1,AZ5P4R54X8Y,UCFi1gCWSp26jr93RptI-srw,youtube,2f772ab3ca91186fa6a48d9a771a53e8891534024ddecb88b7dbc48139388c94,71353,youtube,6e7b66e1b8f9,西方世界与其担心以色列不如担心担心自己，法英美德那么多伊斯兰移民难民看到加沙地带的惨剧肯定会制造更多的本土恐怖袭击。这都是法英美德几百年造孽的报应,2023-10-15 05:20:25.976140
2,-4CIDkIao5I,UCdqGyqo3Brz-T1_GyQ-xrPw,youtube,f65638d94908e60c2506edeb97a95c86ef0bf460181459748a30cc0b8a27500b,72061,youtube,16450772bdb1,God bless you Ron👍,2023-10-09 05:20:25.217489
3,56--kQJslvA,UCdqGyqo3Brz-T1_GyQ-xrPw,youtube,afbf0a48d9bef29bfd5330f218128fe5bff04ef3dce997bd77626c38edc1cb01,72061,youtube,000bfa675b74,Trump is with the arabs.,2023-10-10 05:20:30.714110
4,56--kQJslvA,UCdqGyqo3Brz-T1_GyQ-xrPw,youtube,7da1d45ef956ae8b69528982637d69cd66d16051b3c923fd564333b12dfe466a,72061,youtube,000bfa675b74,USA $6 billion to Iran is working against Israelis today.,2023-10-10 05:20:30.714110
...,...,...,...,...,...,...,...,...,...
7271,zJPLZbrTyVk,UCTotYDdlrxqFBjusZ-QhYzw,youtube,b83db80579622f44487bf8c277ccd97d9d330967f281f74b8a0fe473bb78b161,72056,youtube,dd1ae37e7fba,this is a nice take Hasan! a much bigger improvement compared to the one you had regarding Taiwan 😊,2023-10-11 00:20:17.579115
7272,ZJxW3QCV1Qo,UCqz6ExXs-RA7pRYM3jnxo4Q,youtube,808078e968f4a02e43bc8d6297b0d104d1bb552ad68248644f4fb599315d697d,72145,youtube,4ff1319053ac,Gaza will turn to dust!,2023-10-09 05:20:25.217489
7273,ZJxW3QCV1Qo,UC87scuxEhcJDyrPVPSh4MYA,youtube,92280e5c2e98b417b32353fab83463499d38882ed0dab67cc053c3f3450b0108,72015,youtube,4ff1319053ac,"I think they we exaggerating the deaths in Israeli response, they even haven't properly started yet😢",2023-10-09 05:20:25.217489
7274,ZJxW3QCV1Qo,UCmGjD46m7gryr3D1ydojZlg,youtube,aad99156d47cc7aef125adbe2d9d8cea35b96fe8a059cc89bf5c02b7b2a0ffe3,72048,youtube,4ff1319053ac,I thought uk news was so biased until i went to usa !!! Couldnt believe how biased the news was there and still is .,2023-10-09 05:20:25.217489


In [ ]:
comment_df

,content,type,created_at,updated_at,content_hash,id,skip_computing_sentiment
0,多少都不夠，要有現金流才夠,comment,2023-10-02 00:01:39.566462,2023-10-02 00:01:39.566462,ad6860098adf453271b66e5bfb8a70c444671d6ab4166f5da2957830df824fc2,52218002,0
1,會不會慢慢變成70 75 80歲退休 也太痛苦,comment,2023-10-02 00:01:39.566462,2023-10-02 00:01:39.566462,bacd114d6c4bfebb3e7b582237fa47ca51d7639dba99d30b78982fa15eadbc77,52218003,0
2,至少1千萬 意思是生活最低標準要準備千萬,comment,2023-10-02 00:01:39.566462,2023-10-02 00:01:39.566462,ebefff28d90cd1056038afd671fadd049db1270749415ec508df2c5e21b7a17d,52218004,0
3,20歲的說活到70歲就夠本，但等到自,comment,2023-10-02 00:01:39.566462,2023-10-02 00:01:39.566462,a35d9214ab709934ff93c454fc7e9d4ed1f8bfb9f96427dcb4dc159d947dc686,52218005,0
4,己到40-50歲後，怕早死開始養生，哈,comment,2023-10-02 00:01:39.566462,2023-10-02 00:01:39.566462,9665113b511f3f9659f6cb6cdd47910827b05ee43fbcbc8751dc6619697cdfb8,52218006,0
...,...,...,...,...,...,...,...
320733,Biden..gaza what???..vaganza??..crap..😅😅😅,comment,2023-10-18 13:20:25.943980,2023-10-18 13:20:25.943980,4372e8314c5fb1e26da0296aad020a49d96b88adcacf6db1495515aaf00d5b44,64241923,0
320734,"Stop this crime!! Shame on you for bombing this people who are starving, defenceless and nowhere to run!! Both are murdering innocent children",comment,2023-10-18 13:20:25.943980,2023-10-18 13:20:25.943980,f35fa111be319f284f65e1fc03a0f21f4aa3074273b6e69d1255180c373abbf7,64241924,0
320735,"Actually Biden didn't say occupying Gaza is a mistake.\nBiden answering a question if the US would intervene, he said ""it'd be a mistake'.\nThe media twisted that and say he said the occupation of Gaza is a mistake.",comment,2023-10-18 13:20:25.943980,2023-10-18 13:20:25.943980,ae3f61252792e79455ebc4cf9cf4dee1c2f8d94b856ef90addc225ca75ca9241,64241925,0
320736,"USA coward Biden just doing theatrics and no action ... Blinkens is pleading china and crook dictators for peace ... Biden team wants Bibi to fail and do some settlements with iran Hamas and Hezbollah for sham peace ... Polioticks of next level is on ... muslim countries whom USA supply weapons are passing on their weapons to hamas and Hezbollah ... Muslims, terrorists and dictators are fully united and Christians / democracies are broken records ... ?? .. what settlements with terrorist BBC talking... what about wiping out hamas.... wake up israel its now or never ... coward west is not a friend rather a crook spectator always surrender to terrorist.",comment,2023-10-18 13:20:25.943980,2023-10-18 13:20:25.943980,8606cd47f0ba4cc78debe882cc7569a5115ee3a438d2d278d8b2c5df09f41095,64241926,0
